In [1]:
import numpy as np
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import graphviz
import codecs
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'  

In [2]:
class Node():
    def __init__(self, nodedata, nodeid, wikiid, originaltext):
        self.nodeid = nodeid
        self.nodedata = str(nodedata)
        self.wikiid = wikiid
        self.originaltext = originaltext
        self.state = 'init'
        self.type = 'answer' ## другие возможные типы: quetion, answer, action, external

    

In [19]:
class Relation():
    def __init__(self, parentnodeid, childnodeid, relationdata, activation):
        self.parentnodeid=parentnodeid
        self.childnodeid=childnodeid
        self.activation = activation
        self.relationdata = str(relationdata)
        self.relationType = 'userinput'
    def evaluate(self, answer, question, context):
        self.weight = self.activation(self.relationdata, answer, question, context) 
        return [str(self.childnodeid),  float(self.weight) ]    


In [156]:
class TreeGraph():
    '''
    activation - distance function betven labeldata and answer
    '''
    def __init__(self, activation, threshold):
        self.activation=activation
        self.nodelist = {}
        self.relationlist=defaultdict(list)
        self.threshold = threshold
        self.context={}
        self.nodenumbers={}
    def addNode(self,nodeid, nodedata, wikiid, originaltext):
        self.nodelist[nodeid] = Node(nodedata, nodeid,wikiid,  originaltext)
        self.nodenumbers[nodeid] = len(self.nodelist)
    def addRealtion(self, parentnodeid, childnodeid, relationdata):
        self.relationlist[parentnodeid].append(Relation(parentnodeid, childnodeid, relationdata, self.activation))  
    def evaluateNode(self, nodeid):
        if nodeid in self.relationlist.keys():
            question=self.nodelist[nodeid].nodedata
            for relrel in self.relationlist[nodeid]:
                print(relrel.relationdata)
            answer = input(question + ": ")
            self.context[nodeid] = answer
            tmp=np.zeros((len(self.relationlist[nodeid]), 2),dtype=object)

            for i,relation in enumerate(self.relationlist[nodeid]):
                tmp[i] = relation.evaluate(answer, question, self.context)

            if np.min(tmp[:,1]) >= self.threshold:
                return(self.default())
            n=np.argmin(tmp[:,1])
            nextnodeid=str(tmp[n,0])
            self.evaluateNode(nextnodeid)
        else:
            print(self.nodelist[nodeid].nodedata)
            return(self.nodelist[nodeid].nodedata)
    def default(self):
        print("Я не понял о чем вы")
        #можно попробывать перефразировать вопрос и задать еще раз, или вернуться выше и пойти другой веткой
        return "Я не понял о чем вы"
    def _exporttovis_i(self, result, nodeid, iteration):
        if nodeid in list(self.relationlist.keys()):

            result= '\n' + str(self.nodenumbers[nodeid]) +' [label=\"' + self.nodelist[nodeid].nodedata + '\"] ;'
            iteration +=1
            j=0
            for relrel in self.relationlist[nodeid]:
                result+='\n' + str(self.nodenumbers[nodeid]) +' -> ' + str(self.nodenumbers[relrel.childnodeid]) + \
                ' [headlabel=\"' +relrel.relationdata + '\"];'
                j+=1
                resulttmp, iteration=self._exporttovis_i(result, relrel.childnodeid, iteration)
                result+=resulttmp
                
     
        else:
            result='\n'+str(self.nodenumbers[nodeid]) +' [label=\"' + self.nodelist[nodeid].nodedata + '\"] ;'
            iteration+=1
            return (result, iteration)
        return (result, iteration)
    def exporttovis(self):
        result=''

        tmp=0
        result, tmp=self._exporttovis_i(result, list(self.nodelist.keys())[0],0)
        result = 'digraph Tree {  node [shape=box] ;' + result
        result += ' }'
        file=open("output.txt", "w")
        print(result, file=file)
        file.close()
        return result
    def cleantext(self,text):

        pattern_cleanbegin = re.compile('(^)[\.\s\']+')
        pattern_cleanend = re.compile('[\'\s\.]+($)')
        pattern_wikibold = re.compile('\'{2,}')

        
        text = text.replace('\r', '').replace('\n', '')
        text = pattern_cleanbegin.sub(r'\1', text)
        text = pattern_cleanend.sub(r'\1', text)
        text = pattern_wikibold.sub('', text)
        text = text.replace('\r', '').replace('\n', '')
        text = text.replace('\t', '').replace('\s+', ' ')
        return text
    def filltree(self, filepath, encode):
            
        pattern_firstdigit = re.compile('^\s*(\d+)\.([^\d]*.*)')
        pattern_firstdash = re.compile('^\s*(-|(?:\d+\.\d+))\s*(?:Если)+(.*?),[(?:то)\s]*(.*)')
        pattern_digitincentre = re.compile('.*(?:(?:вопрос)|(?:пункт)).{1,3}?((?:\d+\.\d+)|\d+)')
        pattern_treeend = re.compile('</tree>')
        pattern_treestart = re.compile('<tree>')
        
        
        additionalnode=0
        lastmainnodeid = None
        additionalnodeid=None
        last = 'init'
        treestart = True
        
        with codecs.open(filepath, 'r', encode) as f:
            for line in f:
                line=line.strip()
                if treestart:
                    if pattern_treestart.match(line):
                        treestart=False
                    else continue
                if pattern_treeend.match(line):
                    break
                digitinline = pattern_firstdigit.search(line)
                dashinline = pattern_firstdash.search(line)    
                if digitinline is not None:
                    lastmainnodeid= digitinline.group(1)
                    nodedata = digitinline.group(2)
                    nodedata = self.cleantext(nodedata)
                    self.addNode(digitinline.group(1),nodedata, 0, nodedata)
                    additionalnode=0      
                    last = 'main'
                    continue
                if (dashinline is not None) & (last !="init")  :
                    digitincentre=pattern_digitincentre.search(line)
                    if digitincentre is not None:
                        labeldata = dashinline.group(2)
                        labeldata = self.cleantext( labeldata)
                        self.addRealtion(lastmainnodeid, digitincentre.group(1),labeldata)
                    else:
                        
                        if dashinline.group(1) == '-':
                            additionalnodeid=str(lastmainnodeid) + "_" + str(additionalnode)
                        else:
                            additionalnodeid = dashinline.group(1)
                        nodedata=dashinline.group(3)
                        nodedata = self.cleantext(nodedata)
                        tree.addNode(additionalnodeid, dashinline.group(3), 0, nodedata )

                        labeldata = dashinline.group(2)
                        labeldata = self.cleantext(labeldata)
                
                        tree.addRealtion(lastmainnodeid, additionalnodeid, labeldata)
                
                        additionalnode+=1
                        last='additional'
                    continue
                if last=='main':
                    line =self.cleantext(line)
                    self.nodelist[lastmainnodeid].nodedata = self.nodelist[lastmainnodeid].nodedata + " " + line
                    self.nodelist[lastmainnodeid].originaltext = self.nodelist[lastmainnodeid].nodedata + "\n" + line
            
                if last =='additional':
                    line =self.cleantext(line)
                    self.nodelist[additionalnodeid].nodedata = self.nodelist[additionalnodeid].nodedata + " " + line
                    self.nodelist[additionalnodeid].originaltext = self.nodelist[lastmainnodeid].nodedata + "\n" + line
        

SyntaxError: invalid syntax (<ipython-input-156-204fec98d220>, line 105)

In [ ]:
def activation(template,answer, question, context):
    if re.match('.*' + answer + '.*', template) is not None:
        return 1
    else:
        return 0

In [157]:
def iterative_levenshtein(s, t, nope,nope1, costs=(1, 4, 4)):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
        
        costs: a tuple or a list with three integers (d, i, s)
               where d defines the costs for a deletion
                     i defines the costs for an insertion and
                     s defines the costs for a substitution
    """
    s=s.lower()
    t=t.lower()
    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    
 
    return dist[row][col]


In [158]:
iterative_levenshtein( 'профиль Клиента не заполнен','не заполнен', '111','222')

16

In [159]:
iterative_levenshtein('Нет', 'не', '111','222')

1

In [160]:
tree = TreeGraph(iterative_levenshtein, 40) 
# для начала  расстояние левенштеина, потом можно будет заменить на косинусное расстояние или что нибудь похожее

In [161]:
tree.filltree('WikiAccountBlocked2.txt', 'utf-8-sig')

In [155]:
tree.evaluateNode('1')

Да
Нет
Учетную запись заблокировали после ввода пароля?  : Да
профиль Клиента заполнен
профиль Клиента не заполнен
Необходимо уточнить заполнен ли в АС СББОЛ Профиль Клиента? : не заполнен
Клиент помнит пароль
Клиент не помнит пароль
Уточняем у Клиента помнит ли он свой пароль?  : не помнит
переводим Клиента на линию Операционной поддержки для проведения процедуры аутентификации (8-800-5555-777, далее «6», затем «4») с обязательной передачей информации по Клиенту оператору. 


In [162]:
graph_src = tree.exporttovis()

In [164]:
graph = graphviz.Source(graph_src)  
graph.render('tree')

'tree.pdf'